In [1]:
import pandas as pd
#### Step 2 DATABASE ENGINEERING ####
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Float, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
conn = engine.connect()

In [4]:
#Create a class for the stations table
class Station(Base):
    __tablename__ = 'station'
    station_id = Column(Integer, primary_key = True)
    station= Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
   

In [5]:
#Create a class for the measurements table
class Measurements(Base):
    __tablename__ = 'measurement'
    measurement_id = Column(Integer, primary_key = True)
    station= Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [6]:
Base.metadata.create_all(engine)


In [7]:
# Load the cleaned station csv file into a pandas dataframe
clean_station_df = pd.read_csv('hawaii_stations_clean.csv')
station_data = clean_station_df.to_dict(orient='records')
print(station_data[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [8]:
# Load the cleaned station csv file into a pandas dataframe
clean_measurements_df = pd.read_csv('hawaii_measurements_clean.csv')
#print(clean_measurements_df.head())
measurement_data = clean_measurements_df.to_dict(orient='records')
print(measurement_data[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [9]:
metadata = MetaData(bind=engine)
metadata.reflect()


In [10]:
# Save the reference to the `station` table as a variable called `st`
station_tbl = sqlalchemy.Table('station', metadata, autoload=True)

In [11]:
# Save the reference to the `measurement` table as a variable called `me`
measurement_tbl = sqlalchemy.Table('measurement', metadata, autoload=True)

In [12]:
# Use `st.delete()` to remove any pre-existing data, from station table
conn.execute(station_tbl.delete())

In [13]:
# Use `me.delete()` to remove any pre-existing data, from measurement table
conn.execute(measurement_tbl.delete())

In [14]:
# Use `st.insert()` to insert the data into the station table
# The SQL table is populated during this step
conn.execute(station_tbl.insert(), station_data)

In [15]:
# Use `me.insert()` to insert the data into the measurement table
# The SQL table is populated during this step
conn.execute(measurement_tbl.insert(), measurement_data)

In [16]:
# Test that the insert works by fetching the first 5 rows from station table. 
conn.execute("select * from station limit 5").fetchall()


[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [17]:
#Also verify the row count from the stations table, show match the rows in the clean station CSV file
station_numrows = conn.execute("select count(*) from station;")
print(station_numrows.scalar())

9


In [18]:
# Test that the insert works by fetching the first 5 rows from measurement table. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [19]:
#Also verify the row count from the measurements table, show match the rows in the clean measurement CSV file
measurement_numrows = conn.execute("select count(*) from measurement;")
print(measurement_numrows.scalar())

18103
